<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Usando la API para obtener datos sobre personas desaparecidas del RNPDNO


<p> Julio Waissman Vilanova </p>



<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/RNPDNO-API.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [1]:
import os
import sys
import requests
import datetime

import pandas as pd
import json


## Calentando motores

Para descargar los datos, vamos a consultar directamente la base de datos pública del [Registro Nacional de Personas Desaparecidas y No Localizadas (RNPDNO)](https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index).

El RNPDNO no tiene una API tal cual, sin embargo, [Pablo Reyes Moctezuma](https://github.com/pablorm296) encontró una manera de extraer la información usando la librería `request` de python. La API que, me imagino, el extrajo a punta de prueba y error la documento en [este archivo en markdown](https://github.com/pablorm296/ScrapperRNPDNO/blob/master/Test/API.md). Un chambón.

Vamos air la usando poco a poco, empecemos por tratar de encontrar en el catálogo los indices de estados, municipios y colonias.

In [2]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

Los identificadores de los estados:

In [3]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']
estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


y ahora los municipios de Sonora

In [4]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_id

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
...,...,...
69,65,TUBUTAMA
70,66,URES
71,67,VILLA HIDALGO
72,68,VILLA PESQUEIRA


y por último los identificadores de las colonias del municipio de Hermosillo

In [5]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_COL
DATA = {"idEstado": "26", "idMunicipio": "30"}

r = main_session.post(TARGET_URL, data = DATA)
col_hmo_id = pd.json_normalize(r.json())
col_hmo_id.columns = ['Valor', 'Colonia']
col_hmo_id

,Valor,Colonia
0,0,--TODAS--
1,347025,22 DE SEPTIEMBRE
2,347004,26 DE OCTUBRE
3,347026,4 DE MARZO
4,347027,4 OLIVOS
...,...,...
631,347358,VISTA DEL LAGO
632,347359,Y
633,347582,ZACATON
634,347384,ZAMORA


## Sociodemográficos totales

`Con este `endpoint` se pueden consultar resúmenes generales de la información que se pide. recuerda de revisar los catálogos.

Hay dos variables cuyos valores posibles son los siguientes:

**idEstatusVictima**:
- "0" PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS
- "2" PERSONAS LOCALIZADAS CON VIDA
- "3" PERSONAS LOCALIZADAS SIN VIDA- "4" PERSONAS DESAPARECIDAS
- "5" PERSONAS NO LOCALIZADAS
- "6" PERSONAS LOCALIZADAS
- "7" PERSONAS DESAPARECIDAS Y NO LOCALIZADAS

**idHipotesisNoLocalizacion**:
- "0" --TODAS--
- "1" ACCIDENTE
- "2" CATÁSTROFE
- "3" NO LOCALIZACIÓN VOLUNTARIA
- "4" NO LOCALIZACIÓN INVOLUNTARIA
- "5" SE DESCONOCE

Veamos como funciona pidiendo información de Sonora y de Hermosillo. Empecemos por Sonora

In [6]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"

DATA = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

resumen_sonora = pd.json_normalize(r.json()).T
resumen_sonora.columns = ['Valor']

resumen_sonora


,Valor
TotalGlobal,"7,960"
TotalDesaparecidos,"4,770"
TotalLocalizados,"3,190"
PorcentajeDesaparecidos,59.92 %
PorcentajeLocalizados,40.08 %
TotalSoloDesaparecidos,"4,730"
TotalSoloNoLocalizados,40
PorcentajeSoloDesaparecidos,99.16 %
PorcentajeSoloNoLocalizados,0.84 %
TotalLocalizadosCV,"2,929"


### Ejercicio

Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

In [7]:
# Ya vimos previamente que podemos accesar ala información por estado usando la
# API por Municipios, especificando el query parameter del estado de Sonora, es
# decir, idEstado = 26.

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"
DATA = {
  "idEstado":"26",
  "idMunicipio":"18"
}

# Hacemos la llamada a la API.
r = main_session.post(TARGET_URL, json = DATA)
# Convertimos la respesta a JSON.
res = r.json()

# Podemos primero imprimir el JSON para darnos una idea de donde estanlos valores
# que buscamos.
print(json.dumps(res, indent=2))

{
  "Title": null,
  "Subtitle": null,
  "XAxisCategories": [
    "EJIDAL",
    "AMANECER 1",
    "VILLAS DEL REAL",
    "VALLE DORADO",
    "MONTECARLOS",
    "NOROESTE",
    "LA CORTINA",
    "VILLA BONITA",
    "CIUDAD OBREG\u00d3N",
    "PARQUE INDUSTRIAL",
    "PALMA REAL",
    "ALAMEDA",
    "LAS ESPIGAS",
    "PUENTE DE PICOS",
    "CUMURIPA",
    "CAMPO 47",
    "CAMPO 5",
    "BENITO JU\u00c1REZ",
    "PRIMERO DE MAYO",
    "MORELOS",
    "CASA BLANCA",
    "PASEO ALAMEDA",
    "EL CAMPANARIO",
    "SIN COLONIA DE REFERENCIA",
    "LUIS ECHEVERR\u00cdA ALVAREZ (\u00c1LVARO OBREG\u00d3N)",
    "LAS FLORES",
    "SOCHILOA",
    "JARDINES DEL VALLE",
    "REAL DEL VALLE",
    "CAMPO 30"
  ],
  "XAxisTitle": null,
  "YAxisTitle": "N\u00famero de personas",
  "YAxisTooltipValueSuffix": " personas",
  "TooltipText": null,
  "PointStar": null,
  "Series": [
    {
      "name": "Hombre",
      "data": [
        0,
        0,
        3,
        1,
        1,
        1,
        0,
     

In [8]:
# Los datos se encuentran a manera de series en el subobjeto "Series". Por lo
# tanto, podemos primero obtener de ahi la columna con los datos de mujeres
# desaparecidas. Queremos tambien los nombres de las colonias para ver a cual corresponde cada
# número en la columna de Mujeres Desaparecidas.

mujer_data = []
colonias_data = []

for serie in res["Series"]:
  if serie["name"] == "Mujer":
    mujer_data = serie["data"]

colonias_data = res["XAxisCategories"]

# Creemos ahora el DataFrame.
cajeme_mujeres_df = pd.DataFrame({"Colonias": colonias_data, "Mujeres Desaparecidas": mujer_data})
cajeme_mujeres_df

,Colonias,Mujeres Desaparecidas
0,EJIDAL,1
1,AMANECER 1,1
2,VILLAS DEL REAL,2
3,VALLE DORADO,1
4,MONTECARLOS,0
5,NOROESTE,0
6,LA CORTINA,1
7,VILLA BONITA,2
8,CIUDAD OBREGÓN,4
9,PARQUE INDUSTRIAL,0


## Personas desaparecidas por sexo y colonia

El `endpoint` **BarChartSexoColonia** está diseñado para generar gráficas, pero nos permite extraer información, si la sabemos formatear.

Vamos viendo un ejemplo:


In [9]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"30",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Colonia'] = res['XAxisCategories']
por_colonia = pd.DataFrame(datos)
por_colonia.index = por_colonia.Colonia

por_colonia


,Hombre,Mujer,Indeterminado,Colonia
Colonia,,,,
VILLA HERMOSA,0,4,0,VILLA HERMOSA
MIGUEL ALEMÁN CENTRO,11,6,0,MIGUEL ALEMÁN CENTRO
QUINTA EMILIA,2,0,0,QUINTA EMILIA
ARBOLEDAS,1,1,0,ARBOLEDAS
SAHUARO INDECO,0,2,0,SAHUARO INDECO
UNIVERSIDAD,1,0,0,UNIVERSIDAD
JESÚS GARCIA,7,3,0,JESÚS GARCIA
MESA DEL SERI,1,1,0,MESA DEL SERI
NUEVA PALMIRA,1,0,0,NUEVA PALMIRA


### Ejercicio

¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

In [10]:
# Ya que desconocemos el endpoint especifico para ello, y los parametros no nos
# permiten desglosar por municipio y colonia a la vez, podemos utilizar el mismo
# endpoint que nos da los datos de cada municipio por colonia y llamarlo para
# cada colonia.

# NOTA: Ya que se trata de hacer varias llamadas y procesar las respuestas de
# cada una de ellas, correr esta celda tomará varios segundos.

# Primero, preparamos el URL del endpoint.
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

# Luego, preparamos un DataFrame vacío al cual iremos concatenando el DataFrame
# de cada municipio (igual que hacer un UNION ALL en SQL).
datos = pd.DataFrame({"Hombre": [], "Mujer": [], "Indeterminado": [], "Colonia": [], "Municipio": [], "Municipio_Id": []})

# Iteramos por cada municipio del estado. Ya contamos con los indices gracias a
# un DataFrame previamente obtenido.
for i in range(len(mun_son_id['Valor'])): #mun_son_id['Valor']:
  # Ya que el 0 es para el total de todos los municipios, podemos omitirlo.
  if i == 0: continue

  # Preparamos los parámetros de cada llamada, especificando el ID del municipio
  # del que se trate cada iteración.
  DATA = {
    "idEstado":"26",
    "idMunicipio":str(i),
    "idColonia":"0",
    "idEstatusVictima":"0",
    "idHipotesisNoLocalizacion":"0",
    "idDelito":"0",
    "fechaInicio":"",
    "fechaFin":"",
    "mostrarFechaNula":"0",
    "idNacionalidad":"0",
    "edadInicio":"",
    "edadFin":"",
    "mostrarEdadNula":"0",
    "idHipotesis":"",
    "idMedioConocimiento":"",
    "idCircunstancia":"",
    "tieneDiscapacidad":"",
    "idTipoDiscapacidad":"0",
    "idEtnia":"0",
    "idLengua":"0",
    "idReligion":"",
    "esMigrante":"",
    "idEstatusMigratorio":"0",
    "esLgbttti":"",
    "esServidorPublico":"",
    "esDefensorDH":"",
    "esPeriodista":"",
    "esSindicalista":"",
    "esONG":"",
  }

  # Hacemos la llamada a la API.
  r = main_session.post(TARGET_URL, json = DATA)

  # Obtenemos el JSON del resultado y lo convertimos a un diccionario.
  res = r.json()
  mun_datos = {serie['name']: serie['data'] for serie in res['Series']}
  # Añadimos la columna con el nombre de cada colonia.
  mun_datos['Colonia'] = res['XAxisCategories']
  # Añadimos la columna que indique en cada renglón a qué municipio pertenece.
  mun_datos['Municipio'] = [mun_son_id[mun_son_id['Valor'] == i]['Municipio'].item() for x in range(len(mun_datos['Colonia']))]
  # De la misma manera, añadimos una columna para el ID del municipio.
  mun_datos['Municipio_Id'] = [i for x in range(len(mun_datos['Colonia']))]

  # Finalmente, concatenamos este nuevo DataFrame al DatFrame general que
  # generamos antes del ciclo.
  datos = pd.concat([datos, pd.DataFrame(mun_datos)])


# Añadimos un indice en incremento al DataFrame.
datos.index = [i for i in range(len(datos['Colonia']))]

# Imprimimos los datos de todo el estado, por su colonia y su municipio.
datos

,Hombre,Mujer,Indeterminado,Colonia,Municipio,Municipio_Id
0,2.0,0.0,0.0,ACONCHI,ACONCHI,1.0
1,2.0,3.0,0.0,SIN COLONIA DE REFERENCIA,ACONCHI,1.0
2,0.0,3.0,0.0,VALLES DUARTE,AGUA PRIETA,2.0
3,4.0,6.0,0.0,BUENOS AIRES,AGUA PRIETA,2.0
4,3.0,3.0,0.0,PROGRESO,AGUA PRIETA,2.0
...,...,...,...,...,...,...
421,1.0,0.0,0.0,15 DE MAYO,BENITO JUÁREZ,71.0
422,1.0,0.0,0.0,ROSENDO MONTAÑO,SAN IGNACIO RÍO MUERTO,72.0
423,1.0,0.0,0.0,SAN IGNACIO RIO MUERTO,SAN IGNACIO RÍO MUERTO,72.0
424,14.0,5.0,0.0,SIN COLONIA DE REFERENCIA,SAN IGNACIO RÍO MUERTO,72.0


## Información por sexo y por año

Tambien se puede encontrar información por sexo y por año utilizando otro `endpoint`: **AreaChartSexoAnio**

Sin mas choro, vamos a ver como se usa, otra vez con el estado de Sonora:

In [11]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha_todos = pd.DataFrame(datos)
por_fecha_todos['Fecha'] = pd.to_numeric(por_fecha_todos.Fecha, errors='coerce')
por_fecha_todos.index = por_fecha_todos.Fecha

por_fecha_todos


,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,127,45,5,NaN
1974.0,3,0,0,1974.0
1977.0,1,0,0,1977.0
1978.0,2,0,0,1978.0
1980.0,1,0,0,1980.0
1981.0,7,1,0,1981.0
1982.0,1,0,0,1982.0
1989.0,1,0,0,1989.0
1994.0,2,0,0,1994.0


### Ejercicio

¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

In [12]:
# Solución Particular:

# Si lo que queremos es obtener las desapariciones por sexo para un municipio en
# particular, podemos solo modificar los parámetros de la llamada a la API,
# especificando el municipio que queremos. Por ejemplo, aquí hacemos exactamente
# la misma llamada a la API pero para el municipio de Hermosillo (30).

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"30", # ID de Hermosillo
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha_son = pd.DataFrame(datos)
por_fecha_son['Fecha'] = pd.to_numeric(por_fecha_son.Fecha, errors='coerce')
por_fecha_son.index = por_fecha_son.Fecha

por_fecha_son

,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,24,15,0,NaN
1974.0,1,0,0,1974.0
1977.0,1,0,0,1977.0
1981.0,1,1,0,1981.0
1982.0,1,0,0,1982.0
1995.0,1,0,0,1995.0
2000.0,1,0,0,2000.0
2001.0,1,0,0,2001.0
2002.0,1,0,0,2002.0


In [13]:
# Solución Por Cada Municipio:

# Podemos extraer los datos por fecha de cada municipio y unirlos todos.

# Primero, preparaos la URL para la llamada a la API.
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

# Preparamos un DataFrame vacio para ir concatenando todos los resultados.
datos = pd.DataFrame({"Hombre": [], "Mujer": [], "Indeterminado": [], "Fecha": [], "Municipio": []})

# comenzamos a iterar respecto a cada municipio.
for i in range(len(mun_son_id['Valor'])):
  # Ignoramos el indice 0, ya que son los totales de todos los municipios.
  if i == 0: continue

  # Preparamos los parámetros de cada la llamada con el indice correspondiente.
  DATA = {
    "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
    "idEstado":"26",
    "idMunicipio":str(i),
    "idColonia":"0",
    "edadInicio":"",
    "edadFin":"",
    "mostrarEdadNula":"0",
    "idHipotesisNoLocalizacion":"0",
    "idDelito":"0",
    "idEstatusVictima":"0",
    "fechaInicio":"",
    "fechaFin":"",
    "mostrarFechaNula":"0",
    "idNacionalidad":"0",
    "idHipotesis":"",
    "idMedioConocimiento":"",
    "idCircunstancia":"",
    "idEtnia":"0",
    "idLengua":"0",
    "idReligion":"",
    "tieneDiscapacidad":"",
    "idTipoDiscapacidad":"0",
    "esMigrante":"",
    "idEstatusMigratorio":"0",
    "esLgbttti":"",
    "esServidorPublico":"",
    "esDefensorDH":"",
    "esPeriodista":"",
    "esSindicalista":"",
    "esONG":"",
  }

  # Hacemos la llamada a la API.
  r = main_session.post(TARGET_URL, json = DATA)

  # Pbtenemos el JSON de la respuesta.
  res = r.json()

  # Convertimos la respuesta a un diccionario.
  mun_datos = {serie['name']: serie['data'] for serie in res['Series']}
  # Generamos la columna de las fechas.
  mun_datos['Fecha'] = res['XAxisCategories']
  #Generamos la columna del municipio correspondiente.
  mun_datos['Municipio'] = [mun_son_id[mun_son_id['Valor'] == i]['Municipio'].item() for x in range(len(mun_datos['Fecha']))]

  # Convertimos los datos a un DataFrame.
  por_fecha = pd.DataFrame(mun_datos)
  # Convertimos la columna de fecha al formato numerico para el año.
  por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')

  # Concatenamos los resultados de este municipio al DataFrame general.
  datos = pd.concat([datos, por_fecha])

# Ordenamos el DataFrame final por fecha y luego por municipio.
datos = datos.sort_values(by=['Fecha', 'Municipio'])
# Le damos al DataFrame indices en incremento.
datos.index = [i for i in range(len(datos['Fecha']))]

datos

,Hombre,Mujer,Indeterminado,Fecha,Municipio
0,1.0,0.0,0.0,1974.0,CAJEME
1,1.0,0.0,0.0,1974.0,HERMOSILLO
2,1.0,0.0,0.0,1974.0,SE DESCONOCE
3,1.0,0.0,0.0,1977.0,HERMOSILLO
4,1.0,0.0,0.0,1978.0,NOGALES
...,...,...,...,...,...
557,1.0,0.0,0.0,NaN,PITIQUITO
558,1.0,0.0,0.0,NaN,PUERTO PEÑASCO
559,2.0,3.0,0.0,NaN,SAN LUIS RÍO COLORADO
560,9.0,2.0,5.0,NaN,SE DESCONOCE


### Ejercicio

Extrae alguna información del conjunto de datos que pienses que es relevante, y explica porqué.

In [14]:
# Como ya vimos al sacar las desapariciones por año, a nivel nacional el número
# aumentó considerablemente durenate las últimas décadas. Eso sin mencionar el
# aumento drástico quehubo este último año.

por_fecha_todos

,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,127,45,5,NaN
1974.0,3,0,0,1974.0
1977.0,1,0,0,1977.0
1978.0,2,0,0,1978.0
1980.0,1,0,0,1980.0
1981.0,7,1,0,1981.0
1982.0,1,0,0,1982.0
1989.0,1,0,0,1989.0
1994.0,2,0,0,1994.0


In [15]:
# Delimitando más el problema, podemos ver que este aumento fue igual de
# drástico al considerar solo el estado de Sonora.

por_fecha_son

,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,24,15,0,NaN
1974.0,1,0,0,1974.0
1977.0,1,0,0,1977.0
1981.0,1,1,0,1981.0
1982.0,1,0,0,1982.0
1995.0,1,0,0,1995.0
2000.0,1,0,0,2000.0
2001.0,1,0,0,2001.0
2002.0,1,0,0,2002.0


In [16]:
# Hay noticias y registros que hablan sobre el aumento en desapariciones entre
# el 2017 y el 2020, como se pued ver en los datos. De la misma manera, el
# incremento de este año, 2024, esta llamando la atención alarmantemente.

# Seguramente hay factores externos a los datos que causaron tanto incremento en
# las ultimas décadas, como por ejemnplo la mejora en los métodos de registrar a
# las personas desaparecidas. Eso llevaría a que a lo largo de los años, aunque
# el númeor de desapariciones no aumente en realidad, se vean más personas
# desaparecidas registradas.

# Sin embargo, esto no es necesariamente cierto para los incrementos del 2017 y
# el actual. Hace falta investigar otros factores y contextos para darle sentido
#a estos aumentos repentinos en los datos.

## Practicando a ser investigador de APIs

Ahora te pido que revises si puedes encontrar otros endpoints para recuperar mas información de las bases que no se encuentran liberadas. Puede ser en la misma página, o en blogs o revisando código. Agrega en esta libreta la documentación (o enlaces a dicha documentación) y un ejemplo de uso de una API pobremente documentada.

In [23]:
# Un ejemplo de información no necesariamente liberada es la información de
# mecánicas y reglas del juego de rol de mesa Calabozos y Dragones (DnD por
# Dungeons and Dragons en ingles). La información se encuentra facilmente
# accesible en muchas partes, pero nohay APIs públicas oficiales para toda esta
# información.

# DnD 5e SRD API es una API que expone la información disponible sin problemas
# de copyright, ya que las partes más básicas son open source bajo el reglamento
# de SRD.

# Podemos encontrar documentación en:
#   https://5e-bits.github.io/docs/introduction

# Como un primer ejemplo, podemos solicitar las clases disponibles en el PHB
# (Player's Handbook, el libro básico de reglas de DnD). Para ello, usamos el
# endpoint para "classes/".

DND_API_ROOT = "https://www.dnd5eapi.co/api/"
CLASSES_ROOT = "classes/"

TARGET_URL = DND_API_ROOT + CLASSES_ROOT

dnd_session = requests.Session()
r = dnd_session.get(TARGET_URL)
res = r.json()
datos = res['results']
datos_df = pd.DataFrame(datos)
datos_df.columns = ['Class', 'Name', 'URL']
datos_df

,Class,Name,URL
0,barbarian,Barbarian,/api/classes/barbarian
1,bard,Bard,/api/classes/bard
2,cleric,Cleric,/api/classes/cleric
3,druid,Druid,/api/classes/druid
4,fighter,Fighter,/api/classes/fighter
5,monk,Monk,/api/classes/monk
6,paladin,Paladin,/api/classes/paladin
7,ranger,Ranger,/api/classes/ranger
8,rogue,Rogue,/api/classes/rogue
9,sorcerer,Sorcerer,/api/classes/sorcerer


In [24]:
# Como un ejemplo más especifico, podemos obtener la información específica de
# la clase "Fighter".

dnd_session = requests.Session()
r = dnd_session.get(TARGET_URL + "fighter")
res = r.json()
res

{'index': 'fighter',
 'name': 'Fighter',
 'hit_die': 10,
 'proficiency_choices': [{'desc': 'Choose two skills from Acrobatics, Animal Handling, Athletics, History, Insight, Intimidation, Perception, and Survival',
   'choose': 2,
   'type': 'proficiencies',
   'from': {'option_set_type': 'options_array',
    'options': [{'option_type': 'reference',
      'item': {'index': 'skill-acrobatics',
       'name': 'Skill: Acrobatics',
       'url': '/api/proficiencies/skill-acrobatics'}},
     {'option_type': 'reference',
      'item': {'index': 'skill-animal-handling',
       'name': 'Skill: Animal Handling',
       'url': '/api/proficiencies/skill-animal-handling'}},
     {'option_type': 'reference',
      'item': {'index': 'skill-athletics',
       'name': 'Skill: Athletics',
       'url': '/api/proficiencies/skill-athletics'}},
     {'option_type': 'reference',
      'item': {'index': 'skill-history',
       'name': 'Skill: History',
       'url': '/api/proficiencies/skill-history'}},
    

In [27]:
# Podemos notar que en el JSON tenemos información de distintos factores a
# considerar para un personaje Fighter. Por ejemplo, tenemos la información de
# opciones de proficiencia para tirosde salvación, las opciones de multiclass y
# las proficiencias de armas o equipamento, entre otras.

# Como ejemplo, obtengamos el DataFrame para las proficiencias que obtiene un
# Fighter.

dnd_session = requests.Session()
r = dnd_session.get(TARGET_URL + "fighter")
res = r.json()
fighter_prof_df = pd.DataFrame(res['proficiencies'])
fighter_prof_df.columns = ['Proficiency', 'Name', 'URL']
fighter_prof_df

,Proficiency,Name,URL
0,all-armor,All armor,/api/proficiencies/all-armor
1,shields,Shields,/api/proficiencies/shields
2,simple-weapons,Simple Weapons,/api/proficiencies/simple-weapons
3,martial-weapons,Martial Weapons,/api/proficiencies/martial-weapons
4,saving-throw-str,Saving Throw: STR,/api/proficiencies/saving-throw-str
5,saving-throw-con,Saving Throw: CON,/api/proficiencies/saving-throw-con


In [28]:
# Como se puede ver, esto es una referencia a otra API especifica para la
# información de proficiencias.